In [2]:
# 2023-3-12 custom dataset created by Allen LIN
import torchvision.transforms as trns
from PIL import Image
import pandas as pd
import os
from scipy.io import loadmat
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import numpy as np
import cv2
import torch

class fingerprintDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels.iloc[:, 1] # label of the dataset
        self.data = []
        for i in range(len(self.img_labels)):
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[i, 0])
            image = cv2.imread(img_path)
            self.data.append(image)
        self.data = np.array(self.data)
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = cv2.imread(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
            image = image.cuda()
            #unsqueeze(0)
        if self.target_transform:
            label = self.target_transform(label)
            label = label.cuda()
        return image, label
    def __len__(self):
        return len(self.img_labels)

In [8]:
data = []
img_labels = pd.read_csv("../contact-based_fingerprint/fingerprint_annotations.csv")
img_dir = "../contact-based_fingerprint/first_session/"
print(len(img_labels))
for i in range(len(img_labels)):
    img_path = os.path.join(img_dir, img_labels.iloc[i, 0])
    print(f'img_path: {img_path}')
    image = cv2.imread(img_path)
    data.append(image)
    break
data = np.array(data)
print(type(data))

2016
img_path: ../contact-based_fingerprint/first_session/1_1.jpg
<class 'numpy.ndarray'>


In [54]:
annotations_file = "../contact-based_fingerprint/fingerprint_annotations.csv"
img_dir = "../contact-based_fingerprint/first_session/"
# Normalization for CIFAR
normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                 std=[0.2023, 0.1994, 0.2010])

augmentation = [
    transforms.ToPILImage(), # to PIL format
    transforms.RandomResizedCrop(32, scale=(0.2, 1.0)),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    normalize
]

transform = transforms.Compose(augmentation)
train_dataset = fingerprintDataset(annotations_file, img_dir, transform)
print(train_dataset.data.shape)

train_labels = np.array(train_dataset.targets)
# print(f'train_labels: {train_labels}')
# print(f'train_labels.size: {train_labels.size}')
num_classes = 336
epoch_size = 2016
train_idx = np.array(
    [np.where(train_labels == i)[0][0:int(epoch_size / num_classes)+1] for i in range(0, num_classes+1)], dtype=object).flatten()
train_idx = np.hstack(train_idx)
# train_idx = np.arange(start=1, stop=21, step=1)
# print(f'train_idx: {train_idx}')
# print(f'train_idx.size: {train_idx.size}')
train_dataset.targets = train_labels[train_idx]
print(f'train_dataset.targets: {train_dataset.targets}')
print(f'train_dataset.targets.size: {train_dataset.targets.size}')
print(f'train_dataset.data.size: {train_dataset.data.size}')
train_dataset.data = train_dataset.data[train_idx]

(2016, 356, 328, 3)
train_dataset.targets: [  1   1   1 ... 336 336 336]
train_dataset.targets.size: 2016
train_dataset.data.size: 706212864


In [35]:
# CIFAR-10 dataset
import torchvision.datasets as datasets
import moco.builder
import moco.loader
datadir = './data'
classes = 10
epoch_size = 50000
# Normalization for CIFAR
normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                 std=[0.2023, 0.1994, 0.2010])

augmentation = [
    transforms.RandomResizedCrop(32, scale=(0.2, 1.0)),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    # transforms.RandomApply([moco.loader.GaussianBlur([.1, 2.])], p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    normalize
]

train_dataset = datasets.CIFAR10(root=datadir, train=True, download=True,
                                 transform=moco.loader.TwoCropsTransform(
                                     transforms.Compose(augmentation)))
print(train_dataset.data.shape)

train_labels = np.array(train_dataset.targets)
print(f'train_labels: {train_labels}')
num_classes = classes
train_idx = np.array(
    [np.where(train_labels == i)[0][:int(epoch_size / num_classes)] for i in range(0, num_classes)]).flatten()
train_dataset.targets = train_labels[train_idx]
train_dataset.data = train_dataset.data[train_idx]
print(f'train_idx: {train_idx}')
print(f'train_idx.size: {train_idx.size}')
print(f'train_dataset.targets: {train_dataset.targets}')
print(f'train_dataset.targets.size: {train_dataset.targets.size}')
print(f'train_dataset.data.size: {train_dataset.data.size}')

Files already downloaded and verified
(50000, 32, 32, 3)
train_labels: [6 9 9 ... 9 1 1]
train_idx: [   29    30    35 ... 49963 49971 49997]
train_idx.size: 50000
train_dataset.targets: [0 0 0 ... 9 9 9]
train_dataset.targets.size: 50000
train_dataset.data.size: 153600000
